In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Create a library to change data types
dtype_mapping = {
    "DiagnosisGroupCode": str,
    "DOD": str,
    "ClmAdmitDiagnosisCode": str,
    **{f"ClmDiagnosisCode_{i}": str for i in range(1, 11)}
}

# Identify date columns
date_columns = ["ClaimStartDt", "ClaimEndDt", "DOB", "DOD", "AdmissionDt", "DischargeDt"]

Data loaded successfully.


In [ ]:
# Load the training data and the unlabeled data
train_data = pd.read_csv("data/train_data.csv", dtype=dtype_mapping, parse_dates=date_columns)
unlabeled_data = pd.read_csv("data/unlabeled_data.csv", dtype=dtype_mapping, parse_dates=date_columns)

# Format dates
date_format = "%Y-%m-%d"

# Convert date columns
for col in ["AdmissionDt", "DischargeDt", "DOD"]:
    if col in train_data.columns:
        train_data[col] = pd.to_datetime(train_data[col], format=date_format, errors="coerce")
    if col in unlabeled_data.columns:
        unlabeled_data[col] = pd.to_datetime(unlabeled_data[col], format=date_format, errors="coerce")
print("Data loaded successfully.")

In [1]:
diagnosis_columns = [f"ClmDiagnosisCode_{i}" for i in range(1, 11)]
procedure_columns = [f"ClmProcedureCode_{i}" for i in range(1, 7)]

In [4]:
# Create a function to fix the format of codes
def fix_icd_format(code):
    """Fix ICD-9/10 codes that are incorrectly formatted due to float conversion."""
    if pd.isna(code) or code in ["", "NAN", "NaN", "<NA>"]:
        return pd.NA

    code = str(code).strip().upper()

    if code.endswith(".0"):
        code = code[:-2]

    return code

In [5]:
# Apply the function to fix the format of codes
for col in diagnosis_columns + procedure_columns:
    train_data[col] = train_data[col].replace(
        {"nan": pd.NA, "0nan": pd.NA, "NaN": pd.NA, "NAN": pd.NA, "<NA>": pd.NA, "": pd.NA},
        regex=True
    )

    train_data[col] = train_data[col].astype("string").apply(fix_icd_format)

In [6]:
# Get metadata
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558211 entries, 0 to 558210
Data columns (total 59 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   BeneID                           558211 non-null  object        
 1   ClaimID                          558211 non-null  object        
 2   ClaimStartDt                     558211 non-null  datetime64[ns]
 3   ClaimEndDt                       558211 non-null  datetime64[ns]
 4   Provider                         558211 non-null  object        
 5   InscClaimAmtReimbursed           558211 non-null  int64         
 6   AttendingPhysician               556703 non-null  object        
 7   OperatingPhysician               114447 non-null  object        
 8   OtherPhysician                   199736 non-null  object        
 9   AdmissionDt                      40474 non-null   datetime64[ns]
 10  ClmAdmitDiagnosisCode            145899 non-

In [7]:
# Check head
train_data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,ChronicCount,PotentialFraud,ClaimType
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,1,1,36000,3204,60,70,82,7,Yes,Inpatient
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,1,1,36000,3204,60,70,82,7,No,Inpatient
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,1,1,36000,3204,60,70,82,7,No,Inpatient
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,1,1,5000,1068,250,320,111,6,No,Inpatient
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,2,2,21260,2136,120,100,87,5,No,Inpatient


In [8]:
print(train_data["ClmProcedureCode_1"].dtype)
print(train_data["ClmProcedureCode_1"].unique()[:10])  # Show first 10 unique values

object
[<NA> '7092' '331' '3893' '863' '4576' '9904' '3612' '9672' '9671']


In [9]:
# Create a function to map codes to categories
def map_category(code, category_dict):
    """Maps an ICD-9 code to a category based on provided ranges."""
    if pd.isna(code) or code=="":
        return "Unknown"

    if code.startswith("V"):
        return "V-Codes"
    if code.startswith("E"):
        return "E-Codes"

    code = int(code)

    for (low, high), category in category_dict.items():
        if low*100 <= code <= high*100:
            return category

    return "Unknown"

In [10]:
# Create dictionary for mapping diagnosis categories
diagnosis_category_map = {
    (1, 139): "Infectious & Parasitic Diseases",
    (140, 239): "Neoplasms",
    (240, 279): "Endocrine, Nutritional, and Metabolic Diseases",
    (280, 289): "Diseases of the Blood",
    (290, 319): "Mental Disorders",
    (320, 389): "Diseases of the Nervous System and Sense Organs",
    (390, 459): "Diseases of the Circulatory System",
    (460, 519): "Diseases of the Respiratory System",
    (520, 579): "Diseases of the Digestive System",
    (580, 629): "Diseases of the Genitourinary System",
    (630, 679): "Pregnancy, Childbirth, and the Puerperium",
    (680, 709): "Diseases of the Skin and Subcutaneous Tissue",
    (710, 739): "Diseases of the Musculoskeletal System",
    (740, 759): "Congenital Anomalies",
    (780, 799): "Symptoms, Signs, and Ill-Defined Conditions",
    (800, 999): "Injury and Poisoning",
}

# Create new diagnosis category columns
for col in diagnosis_columns:
    train_data[col] = train_data[col].astype(str).replace({"<NA>": ""}).str.strip().str.upper()
    train_data[f"{col}_Category"] = train_data[col].apply(lambda x: map_category(x, diagnosis_category_map))

In [11]:
# Create dictionary for mapping procedure categories
procedure_category_map = {
    (0, 0): "Miscellaneous Diagnostic & Therapeutic Procedures",
    (1, 5): "Procedures on the Nervous System",
    (6, 7): "Procedures on the Endocrine System",
    (8, 16): "Procedures on the Eye & Ear",
    (17, 20): "Operations on the Cardiovascular System",
    (21, 29): "Operations on the Respiratory System",
    (30, 34): "Operations on the Digestive System",
    (35, 39): "Cardiovascular Procedures",
    (40, 41): "Procedures on the Lymphatic & Hemic System",
    (42, 54): "Procedures on the Digestive System",
    (55, 59): "Procedures on the Urinary System",
    (60, 64): "Procedures on the Male Genital Organs",
    (65, 71): "Procedures on the Female Genital Organs",
    (72, 75): "Obstetric & Gynecological Procedures",
    (76, 84): "Orthopedic Procedures",
    (85, 86): "Operations on the Breast and Skin",
    (87, 99): "Radiology, Physical Therapy, and Other Miscellaneous Procedures",
}

# Create new procedure category columns
for col in procedure_columns:
    train_data[col] = train_data[col].astype(str).replace({"<NA>": ""}).str.strip().str.upper()
    train_data[f"{col}_Category"] = train_data[col].apply(lambda x: map_category(x, procedure_category_map))

In [12]:
# Create a primary procedure column
train_data["PrimaryProcedure"] = (
    train_data["ClmProcedureCode_1"]
    .astype(str)
    .str.strip()
    .replace({"nan": pd.NA, "0nan": pd.NA, "NaN": pd.NA, "": pd.NA})
    .str.zfill(4)
)

train_data["PrimaryProcedure"] = train_data["PrimaryProcedure"].fillna("Unknown")

In [13]:
# Create a number of procedures column
train_data["NumProcedures"] = train_data[procedure_columns].notna().sum(axis=1)

In [14]:
# Read in procedure descriptions
proc_desc = pd.read_excel("medical_codes/CMS32_DESC_LONG_SHORT_SG.xlsx", dtype=str)

proc_desc.rename(columns={
    "PROCEDURE CODE": "ProcedureCode",
    "LONG DESCRIPTION": "PrimaryProcedure_LongDesc",
    "SHORT DESCRIPTION": "PrimaryProcedure_ShortDesc"
}, inplace=True)
proc_desc["ProcedureCode"] = proc_desc["ProcedureCode"].astype(str).str.zfill(4)

# Merge procedure descriptions to training data
train_data = train_data.merge(proc_desc, left_on="PrimaryProcedure", right_on="ProcedureCode", how="left")

train_data.drop(columns=["ProcedureCode"], inplace=True, errors="ignore")

print("Merged procedure descriptions.")

Merged procedure descriptions.


In [15]:
# Read in diagnosis descriptions
dx_desc = pd.read_excel("medical_codes/CMS32_DESC_LONG_SHORT_DX.xlsx", dtype=str)
dx_desc.rename(columns={"DIAGNOSIS CODE": "DiagnosisCode"}, inplace=True)
dx_desc["DiagnosisCode"] = dx_desc["DiagnosisCode"].astype(str).str.zfill(4)

# Merge diagnosis descriptions to training data
for col in diagnosis_columns:
    if col in train_data.columns:
        train_data = train_data.merge(dx_desc, left_on=col, right_on="DiagnosisCode", how="left", suffixes=("", f"_{col}"))

print("Merged diagnosis descriptions.")

Merged diagnosis descriptions.


In [16]:
# Remove unneeded columns
columns_to_drop = [
    "DiagnosisGroupCode",
    *diagnosis_columns,
    *[f"ClmProcedureCode_{i}" for i in range(1, 7)],
    "DiagnosisCode",
    *[f"DiagnosisCode_ClmDiagnosisCode_{i}" for i in range(1, 11)]
]
train_data.drop(columns=[col for col in columns_to_drop if col in train_data.columns], inplace=True, errors="ignore")

In [17]:
# Rename diagnosis code columns
for i in range(1, 11):
    if f"LONG DESCRIPTION_ClmDiagnosisCode_{i}" in train_data.columns:
        train_data.rename(columns={
            f"LONG DESCRIPTION_ClmDiagnosisCode_{i}": f"ClmDiagnosisCode_{i}_LongDesc",
            f"SHORT DESCRIPTION_ClmDiagnosisCode_{i}": f"ClmDiagnosisCode_{i}_ShortDesc"
        }, inplace=True)

if "LONG DESCRIPTION" in train_data.columns:
    train_data.rename(columns={"LONG DESCRIPTION": "ClmDiagnosisCode_1_LongDesc"}, inplace=True)
if "SHORT DESCRIPTION" in train_data.columns:
    train_data.rename(columns={"SHORT DESCRIPTION": "ClmDiagnosisCode_1_ShortDesc"}, inplace=True)

In [18]:
# Create column for number of unknown procedures
num_diagnosis_desc = 10

unknown_procedure_count = (train_data[["PrimaryProcedure_LongDesc", "PrimaryProcedure_ShortDesc"]] == "Unknown").sum(axis=1)
unknown_diagnosis_count = train_data[[f"ClmDiagnosisCode_{i}_LongDesc" for i in range(1, 11)]].apply(lambda row: (row == "Unknown").sum(), axis=1)

# Create flag for more than 0 unknown procedures
train_data["Flag_Unknown_Procedures"] = unknown_procedure_count > 1
train_data["Flag_Unknown_Diagnoses"] = unknown_diagnosis_count > (num_diagnosis_desc / 2)

print(f"{train_data['Flag_Unknown_Procedures'].sum()} records flagged for unknown procedure descriptions.")
print(f"{train_data['Flag_Unknown_Diagnoses'].sum()} records flagged for unknown diagnosis descriptions.")

0 records flagged for unknown procedure descriptions.
0 records flagged for unknown diagnosis descriptions.


In [19]:
# Check metadata
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559877 entries, 0 to 559876
Data columns (total 84 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   BeneID                           559877 non-null  object        
 1   ClaimID                          559877 non-null  object        
 2   ClaimStartDt                     559877 non-null  datetime64[ns]
 3   ClaimEndDt                       559877 non-null  datetime64[ns]
 4   Provider                         559877 non-null  object        
 5   InscClaimAmtReimbursed           559877 non-null  int64         
 6   AttendingPhysician               558367 non-null  object        
 7   OperatingPhysician               115736 non-null  object        
 8   OtherPhysician                   200081 non-null  object        
 9   AdmissionDt                      41685 non-null   datetime64[ns]
 10  ClmAdmitDiagnosisCode            147187 non-

In [20]:
# Check head
train_data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7_LongDesc,ClmDiagnosisCode_7_ShortDesc,ClmDiagnosisCode_8_LongDesc,ClmDiagnosisCode_8_ShortDesc,ClmDiagnosisCode_9_LongDesc,ClmDiagnosisCode_9_ShortDesc,ClmDiagnosisCode_10_LongDesc,ClmDiagnosisCode_10_ShortDesc,Flag_Unknown_Procedures,Flag_Unknown_Diagnoses
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,Other and unspecified hyperlipidemia,Hyperlipidemia NEC/NOS,Secondary malignant neoplasm of other specifie...,Secondary malig neo NEC,"Acute kidney failure, unspecified",Acute kidney failure NOS,NaN,NaN,False,False
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,"Diabetes with neurological manifestations, typ...",DMII neuro uncntrld,"Hypertensive chronic kidney disease, unspecifi...",Hy kid NOS w cr kid I-IV,Unspecified essential hypertension,Hypertension NOS,NaN,NaN,False,False
4,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,"Diabetes with neurological manifestations, typ...",DMII neuro uncntrld,"Hypertensive chronic kidney disease, unspecifi...",Hy kid NOS w cr kid I-IV,Unspecified essential hypertension,Hypertension NOS,NaN,NaN,False,False


In [21]:
# Write cleaned data
train_data.to_csv("data/train_data_cleaned.csv", index=False)